# CLV EDA & PNBD Baseline

This is a starter notebook. We'll fill it in Step 2.


In [3]:
from pathlib import Path
import pandas as pd, shutil

PROJ = Path.cwd().parents[0] if Path.cwd().name == "notebooks" else Path.cwd()
RAW  = PROJ / "data" / "raw"
PROC = PROJ / "data" / "processed"
RAW.mkdir(parents=True, exist_ok=True)
PROC.mkdir(parents=True, exist_ok=True)

candidates = [
    RAW / "online_retail_ii.csv",                
    Path(r"C:\Users\balla\online_retail_II.csv"),
    Path("/c/Users/balla/online_retail_II.csv"), 
]
src = next((p for p in candidates if p.exists()), None)
if src is None:
    raise FileNotFoundError("I couldn't find online_retail_II.csv. Put it in data/raw/ or C:\\Users\\balla and rerun.")

dst = RAW / "online_retail_ii.csv"
if src != dst:
    shutil.copy2(src, dst)

data_path = dst
print("Using:", data_path)

# load 
df = pd.read_csv(data_path)

ren = {}
for want in ["InvoiceNo","StockCode","Description","Quantity","InvoiceDate","UnitPrice","CustomerID","Country","Price"]:
    for c in df.columns:
        if c.lower() == want.lower():
            ren[c] = want
df = df.rename(columns=ren)
if "UnitPrice" not in df.columns and "Price" in df.columns:
    df["UnitPrice"] = df["Price"]

df.head()
df.info()


Using: C:\Users\balla\Downloads\clv-uplift-optimizer-starter\clv-uplift-optimizer\data\raw\online_retail_ii.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      525461 non-null  object 
 1   StockCode    525461 non-null  object 
 2   Description  522533 non-null  object 
 3   Quantity     525461 non-null  int64  
 4   InvoiceDate  525461 non-null  object 
 5   Price        525461 non-null  float64
 6   Customer ID  417534 non-null  float64
 7   Country      525461 non-null  object 
 8   UnitPrice    525461 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 36.1+ MB


In [4]:
import pandas as pd

# Harmonize column names from your CSV variant
rename_map = {}
if "Customer ID" in df.columns and "CustomerID" not in df.columns:
    rename_map["Customer ID"] = "CustomerID"
if "Invoice" in df.columns and "InvoiceNo" not in df.columns:
    rename_map["Invoice"] = "InvoiceNo"
# Keep UnitPrice as the canonical price column
if "UnitPrice" not in df.columns and "Price" in df.columns:
    rename_map["Price"] = "UnitPrice"
df = df.rename(columns=rename_map)

# Parse/convert types
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")
df["Quantity"]    = pd.to_numeric(df["Quantity"], errors="coerce")
df["UnitPrice"]   = pd.to_numeric(df["UnitPrice"], errors="coerce")
# CustomerID often arrives as float (13085.0). Make it a proper ID (string).
df["CustomerID"]  = pd.to_numeric(df["CustomerID"], errors="coerce")

# Core cleaning
# Remove cancellations/returns (InvoiceNo starting with 'C')
if "InvoiceNo" in df.columns:
    df["InvoiceNo"] = df["InvoiceNo"].astype(str)
    df = df[~df["InvoiceNo"].str.startswith("C")]

# Drop rows without a valid customer or date
df = df.dropna(subset=["CustomerID", "InvoiceDate"])

# Positive quantity and price only
df = df[(df["Quantity"] > 0) & (df["UnitPrice"] > 0)]

# Convert CustomerID to string (no decimals)
df["CustomerID"] = df["CustomerID"].astype("int64").astype(str)


df["Sales"] = df["Quantity"] * df["UnitPrice"]

n_rows      = len(df)
n_customers = df["CustomerID"].nunique()
date_min    = df["InvoiceDate"].min()
date_max    = df["InvoiceDate"].max()
print(f"Rows after clean: {n_rows:,} | Customers: {n_customers:,} | Period: {date_min:%Y-%m-%d} → {date_max:%Y-%m-%d}")
print("Revenue total:", round(df["Sales"].sum(), 2))


keep_cols = ["InvoiceNo","StockCode","Description","Quantity","InvoiceDate","UnitPrice","CustomerID","Country","Sales"]
df = df[keep_cols].copy()


(PROC / "transactions_clean.csv").parent.mkdir(parents=True, exist_ok=True)
df.to_csv(PROC / "transactions_clean.csv", index=False)
print("Saved:", PROC / "transactions_clean.csv")

df.head(3)


Rows after clean: 407,664 | Customers: 4,312 | Period: 2009-12-01 → 2010-12-09
Revenue total: 8832003.27
Saved: C:\Users\balla\Downloads\clv-uplift-optimizer-starter\clv-uplift-optimizer\data\processed\transactions_clean.csv


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0


In [5]:
import sys, pkgutil, subprocess, numpy as np, pandas as pd
if not pkgutil.find_loader("lifetimes"):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "lifetimes"])

from lifetimes.utils import summary_data_from_transaction_data, calibration_and_holdout_data
from lifetimes import BetaGeoFitter, GammaGammaFitter

# Lifetimes summary (frequency, recency, T, monetary_value)
summary = summary_data_from_transaction_data(
    df, customer_id_col="CustomerID", datetime_col="InvoiceDate",
    monetary_value_col="Sales", observation_period_end=None, freq="D"
)
print("Summary shape:", summary.shape)

# Time-based 80/20 split (calibration/holdout)
start, end = df["InvoiceDate"].min(), df["InvoiceDate"].max()
cutoff = start + (end - start) * 0.8
coh = calibration_and_holdout_data(
    df, customer_id_col="CustomerID", datetime_col="InvoiceDate",
    monetary_value_col="Sales",
    calibration_period_end=cutoff.normalize() + pd.Timedelta(days=1) - pd.Timedelta(seconds=1),
    observation_period_end=None, freq="D",
)
print("Cal/Holdout shape:", coh.shape)

# Fit BG/NBD and evaluate on holdout
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(summary["frequency"], summary["recency"], summary["T"])

exp = bgf.conditional_expected_number_of_purchases_up_to_time(
    coh["duration_holdout"].values,
    coh["frequency_cal"].values,
    coh["recency_cal"].values,
    coh["T_cal"].values,
)
act = coh["frequency_holdout"].values
eval_stats = {
    "customers": int(len(act)),
    "mae": float(np.mean(np.abs(exp - act))),
    "rmse": float(np.sqrt(np.mean((exp - act)**2))),
    "mape": float(np.mean(np.where(act>0, np.abs((exp-act)/act), 0))),
    "exp_mean": float(np.mean(exp)),
    "act_mean": float(np.mean(act)),
}
print("Holdout eval:", eval_stats)

# Fit Gamma–Gamma for AOV if enough repeaters
repeaters = summary[(summary["frequency"] > 0) & (summary["monetary_value"] > 0)]
ggf = None
if len(repeaters) >= 30:
    ggf = GammaGammaFitter()
    ggf.fit(repeaters["frequency"], repeaters["monetary_value"])
    print("Gamma-Gamma fitted on", len(repeaters), "customers")
else:
    print("Skipping Gamma-Gamma (repeaters <", len(repeaters), ")")

# Compute 12-month CLV
h = 365
exp_purch = bgf.conditional_expected_number_of_purchases_up_to_time(
    h, summary["frequency"], summary["recency"], summary["T"]
)
if ggf is not None:
    exp_aov = ggf.conditional_expected_average_profit(
        summary["frequency"], summary["monetary_value"].fillna(0)
    )
else:
    exp_aov = summary["monetary_value"].fillna(summary["monetary_value"].median())

clv = summary.copy()
clv[f"exp_purchases_h{h}"] = exp_purch
clv["exp_aov"] = exp_aov
clv[f"exp_clv_h{h}"] = clv[f"exp_purchases_h{h}"] * clv["exp_aov"]
clv["exp_clv_disc"] = clv[f"exp_clv_h{h}"] / 1.01  # simple discount placeholder

# Save artifacts 
summary.to_csv(PROC / "lifetimes_summary.csv", index=False)
coh.to_csv(PROC / "clv_calibration_holdout.csv", index=False)
pd.DataFrame([eval_stats]).to_csv(PROC / "clv_holdout_eval.csv", index=False)
clv.to_csv(PROC / "clv_h365.csv", index=False)
print("Saved to:", PROC)

# Top CLV customers
clv.sort_values("exp_clv_disc", ascending=False).head(10)


C:\Users\balla\AppData\Local\Temp\ipykernel_23604\4259540355.py:2: DeprecationWarning: 'pkgutil.find_loader' is deprecated and slated for removal in Python 3.14; use importlib.util.find_spec() instead
  if not pkgutil.find_loader("lifetimes"):


Summary shape: (4312, 4)
Cal/Holdout shape: (3498, 7)
Holdout eval: {'customers': 3498, 'mae': inf, 'rmse': inf, 'mape': inf, 'exp_mean': inf, 'act_mean': 1.1415094339622642}
Gamma-Gamma fitted on 2818 customers
Saved to: C:\Users\balla\Downloads\clv-uplift-optimizer-starter\clv-uplift-optimizer\data\processed


C:\Users\balla\Downloads\clv-uplift-optimizer-starter\clv-uplift-optimizer\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\balla\AppData\Local\Temp\ipykernel_23604\4259540355.py:41: RuntimeWarning: divide by zero encountered in divide
  "mape": float(np.mean(np.where(act>0, np.abs((exp-act)/act), 0))),
C:\Users\balla\Downloads\clv-uplift-optimizer-starter\clv-uplift-optimizer\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,frequency,recency,T,monetary_value,exp_purchases_h365,exp_aov,exp_clv_h365,exp_clv_disc
CustomerID,,,,,,,,
12370,1.0,44.0,303.0,211.66,inf,326.064582,inf,inf
18286,1.0,247.0,358.0,833.48,inf,610.176447,inf,inf
18275,1.0,153.0,366.0,369.21,inf,398.049762,inf,inf
12347,1.0,37.0,39.0,711.79,inf,554.575835,inf,inf
18271,1.0,122.0,366.0,488.30,inf,452.462424,inf,inf
12352,1.0,17.0,27.0,200.05,inf,320.759930,inf,inf
18239,1.0,50.0,57.0,438.10,inf,429.525858,inf,inf
18230,1.0,147.0,160.0,1192.86,inf,774.378502,inf,inf
12371,1.0,236.0,280.0,1571.74,inf,947.490179,inf,inf


In [6]:
# Stabilize BG/NBD and recompute eval + CLV
import numpy as np
from lifetimes import BetaGeoFitter, GammaGammaFitter

def fit_bgnbd_stable(summary, penalizers=(0.05, 0.1, 0.2, 0.5, 1.0, 2.0)):
    """Trying heavier regularization until 'a' > 1 for stable expectations."""
    chosen = None
    for p in penalizers:
        m = BetaGeoFitter(penalizer_coef=p)
        m.fit(summary["frequency"], summary["recency"], summary["T"])
        params = m.params_.to_dict()
        print(f"penalizer={p:.3g} → r={params['r']:.3f}, alpha={params['alpha']:.3f}, a={params['a']:.3f}, b={params['b']:.3f}")
        if params["a"] > 1.01:
            chosen = m
            break
    return chosen or m

# Refit BG/NBD with stability
bgf = fit_bgnbd_stable(summary)

# Re-evaluate holdout; ignore any non-finite rows if they remain
exp_h = bgf.conditional_expected_number_of_purchases_up_to_time(
    coh["duration_holdout"].values,
    coh["frequency_cal"].values,
    coh["recency_cal"].values,
    coh["T_cal"].values,
)
act_h = coh["frequency_holdout"].values
mask  = np.isfinite(exp_h)
eval_stats2 = {
    "customers_finite": int(mask.sum()),
    "mae": float(np.mean(np.abs(exp_h[mask] - act_h[mask]))),
    "rmse": float(np.sqrt(np.mean((exp_h[mask] - act_h[mask])**2))),
    "exp_mean": float(np.mean(exp_h[mask])),
    "act_mean": float(np.mean(act_h[mask])),
}
print("Holdout eval (finite only):", eval_stats2)

# CLV horizon: try 365d
def exp_purchases_h(h):
    e = bgf.conditional_expected_number_of_purchases_up_to_time(
        h, summary["frequency"], summary["recency"], summary["T"]
    )
    return e, np.isfinite(e).all()

exp_purch, ok = exp_purchases_h(365)
h_used = 365
if not ok:
    exp_purch, ok = exp_purchases_h(180)
    h_used = 180
    print("Using 180-day horizon for stability.")

# AOV via Gamma-Gamma
try:
    ggf
except NameError:
    rep = summary[(summary["frequency"] > 0) & (summary["monetary_value"] > 0)]
    ggf = None
    if len(rep) >= 30:
        ggf = GammaGammaFitter()
        ggf.fit(rep["frequency"], rep["monetary_value"])

exp_aov = (
    ggf.conditional_expected_average_profit(
        summary["frequency"], summary["monetary_value"].fillna(0)
    ) if ggf is not None else summary["monetary_value"].fillna(summary["monetary_value"].median())
)

clv2 = summary.copy()
clv2[f"exp_purchases_h{h_used}"] = exp_purch
clv2["exp_aov"] = exp_aov
clv2[f"exp_clv_h{h_used}"] = clv2[f"exp_purchases_h{h_used}"] * clv2["exp_aov"]
clv2["exp_clv_disc"] = clv2[f"exp_clv_h{h_used}"] / 1.01

# Save the stabilized outputs
clv2.to_csv(PROC / f"clv_h{h_used}_stable.csv", index=False)
print("Saved:", PROC / f"clv_h{h_used}_stable.csv")

clv2.sort_values("exp_clv_disc", ascending=False).head(10)


penalizer=0.05 → r=0.699, alpha=57.430, a=0.000, b=0.000
penalizer=0.1 → r=0.616, alpha=50.529, a=0.000, b=0.000
penalizer=0.2 → r=0.524, alpha=42.906, a=0.000, b=0.000
penalizer=0.5 → r=0.405, alpha=32.894, a=0.000, b=0.000
penalizer=1 → r=0.324, alpha=26.028, a=0.000, b=0.000
penalizer=2 → r=0.253, alpha=20.077, a=0.000, b=0.000
Holdout eval (finite only): {'customers_finite': 3498, 'mae': 0.8522365812814707, 'rmse': 1.395685139160796, 'exp_mean': 0.8255679756390332, 'act_mean': 1.1415094339622642}
Using 180-day horizon for stability.
Saved: C:\Users\balla\Downloads\clv-uplift-optimizer-starter\clv-uplift-optimizer\data\processed\clv_h180_stable.csv


C:\Users\balla\Downloads\clv-uplift-optimizer-starter\clv-uplift-optimizer\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,frequency,recency,T,monetary_value,exp_purchases_h180,exp_aov,exp_clv_h180,exp_clv_disc
CustomerID,,,,,,,,
18102,42.0,373.0,373.0,8259.002619,19.348862,8043.319950,155629.090165,154088.208085
14646,45.0,363.0,372.0,5393.010222,20.775492,5265.091449,109384.863081,108301.844635
14156,78.0,367.0,373.0,2506.409487,35.834172,2475.126575,88694.111826,87815.952303
14911,127.0,373.0,373.0,1192.235748,58.272511,1185.096540,69058.550668,68374.802641
13694,48.0,362.0,370.0,2685.693750,22.266353,2630.999079,58582.753474,58002.726212
16754,18.0,269.0,276.0,3505.892778,11.097095,3314.879698,36785.535830,36421.322604
17511,24.0,370.0,372.0,3393.022083,11.134534,3252.835146,36218.803378,35860.201364
15061,40.0,371.0,373.0,2000.241000,18.433012,1954.704256,36031.086530,35674.343099
16684,18.0,353.0,367.0,4407.782778,8.488221,4160.901918,35318.656838,34968.967167


In [8]:
# Finalize CLV outputs (top customers + tidy scores)
from pathlib import Path
import pandas as pd

scores = (
    clv2.reset_index()[["CustomerID", f"exp_purchases_h{h_used}", "exp_aov", f"exp_clv_h{h_used}", "exp_clv_disc"]]
    .rename(columns={
        f"exp_purchases_h{h_used}": "exp_purchases",
        f"exp_clv_h{h_used}": "exp_clv"
    })
    .sort_values("exp_clv_disc", ascending=False)
)

# save tidy scores + a top-500 list
scores.to_csv(PROC / f"clv_scores_h{h_used}.csv", index=False)
scores.head(500).to_csv(PROC / f"clv_top500_h{h_used}.csv", index=False)
print("Saved:", PROC / f"clv_scores_h{h_used}.csv")
print("Saved:", PROC / f"clv_top500_h{h_used}.csv")

# quick sanity peek
scores.head(10)


Saved: C:\Users\balla\Downloads\clv-uplift-optimizer-starter\clv-uplift-optimizer\data\processed\clv_scores_h180.csv
Saved: C:\Users\balla\Downloads\clv-uplift-optimizer-starter\clv-uplift-optimizer\data\processed\clv_top500_h180.csv


,CustomerID,exp_purchases,exp_aov,exp_clv,exp_clv_disc
4183,18102,19.348862,8043.319950,155629.090165,154088.208085
1637,14646,20.775492,5265.091449,109384.863081,108301.844635
1269,14156,35.834172,2475.126575,88694.111826,87815.952303
1840,14911,58.272511,1185.096540,69058.550668,68374.802641
939,13694,22.266353,2630.999079,58582.753474,58002.726212
3177,16754,11.097095,3314.879698,36785.535830,36421.322604
3744,17511,11.134534,3252.835146,36218.803378,35860.201364
1951,15061,18.433012,1954.704256,36031.086530,35674.343099
3128,16684,8.488221,4160.901918,35318.656838,34968.967167
4065,17949,19.398212,1360.394142,26389.213865,26127.934519
